# Tutorial 2 for Python

## Make a scenario of Dantzig's Transport Problem using the *ix modeling platform* (ixmp)

<img style="float: right; height: 80px;" src="_static/python.png">

### Aim and scope of the tutorial

This tutorial uses teh transport problem scenario developed in the first tutorial and illustrates how the ixmp framework can be applied for scenario analysis in the sense often used in economic or environmental modeling: develop a baseline, create a clone from the baseline with altered parameters or assumptions, and solve the new model. Then, compare the results from the original and altered scenario versions.

In particular, this tutorial will take you through the following steps:
0. Launch an ixmp.Platform instance and retrieve the ixmp.Scenario instance of Dantzig's transport problem
0. Retrieve some data from the scenario for illustration of filters
0. Make a clone of the baseline scenario, then check out the clone and make changes:  
   in this case, add a new demand location and transport costs to that city
0. Solve the new scenario
0. Display the solution of both the baseline and the new scenario

### Launching the platform and loading a scenario from the ixmp database instance

We launch a platform instance and display all models/scenarios currently stored in the connected database instance.

In [ ]:
# Import required packages
import ixmp

In [ ]:
# launch the ix modeling platform using the default local database
mp = ixmp.Platform()

In [ ]:
# Model and scenario name for Dantzig's transport problem
model = "canning problem"
scenario = "standard"

In [ ]:
# List all Scenarios in the database
mp.scenario_list()

If you have just run the first, ``py_transport`` tutorial, the existing scenario should appear, and we can load it.
Uncomment and run the following line.

In [ ]:
# # Load the default version of the scenario created in the first tutorial
# scen = ixmp.Scenario(mp, model, scenario)

If not (e.g. starting with this tutorial), we can use a function that creates the scenario from scratch in one step:

In [ ]:
from ixmp.testing import make_dantzig

scen = make_dantzig(mp, solve=".")

### Retrieve some data from the scenario for illustration of filters

Before cloning a scenario and editing data, this section illustrates two-and-a-half methods to retrieve data for a parameter from a scenario.

In [ ]:
# load the distance parameter
d = scen.par("d")
d

In [ ]:
# Show only the distances for connections from Seattle by filtering the pandas.DataFrame
# returned above
d[d["i"] == "seattle"]

In [ ]:
# for faster access or more complex filtering,
# it may be easier to only load specific parameter elements using a dictionary
ele_filter = {}
ele_filter["i"] = ["seattle"]
ele_filter["j"] = ["chicago", "topeka"]

d_filtered = scen.par("d", ele_filter)
d_filtered

### Make a clone of the baseline scenario, then check out the clone and edit the scenario

For illustration of a scenario analysis workflow, we add a new demand location ``detroit`` and add a demand level and transport costs to that city.
Because the production capacity does not allow much slack for increased production, we also reduce the demand level in ``chicago``.

In [ ]:
# create a new scenario by cloning the scenario (without keeping the solution)
scen_detroit = scen.clone(
    model=model,
    scenario="detroit",
    annotation="extend the Transport problem by a new city",
    keep_solution=False,
)

In [ ]:
# check out the scenario to make changes
scen_detroit.check_out()

In [ ]:
# reduce demand in chicago
scen_detroit.add_par("b", "chicago", 200, "cases")

# add a new city with demand and distances
scen_detroit.add_set("j", "detroit")
scen_detroit.add_par("b", "detroit", 150, "cases")
scen_detroit.add_par("d", ["seattle", "detroit"], 1.7, "cases")
scen_detroit.add_par("d", ["san-diego", "detroit"], 1.9, "cases")

In [ ]:
# commit changes to the ixmp database instance
comment = "add new city 'detroit' with demand, reduce demand in 'chicago'"
scen_detroit.commit(comment)
scen_detroit.set_as_default()

### Solve the new scenario

In [ ]:
scen_detroit.solve(model="dantzig")

### Display and analyze the results

For comparison between the baseline scenario, i.e., the original transport problem, and the "detroit" scenario, we show the solution for both cases.

In [ ]:
# display the objective value of the solution in the baseline scenario
scen.var("z")

In [ ]:
# display the objective value of the solution in the "detroit" scenario
scen_detroit.var("z")

In [ ]:
# Display the quantities transported from canning plants to demand locations in the
# baseline scenario
scen.var("x")

In [ ]:
# Display the quantities transported from canning plants to demand locations in the
# "detroit" scenario
scen_detroit.var("x")

In [ ]:
# Display the quantities and marginals (=shadow prices) of the demand balance
# constraints in the baseline scenario
scen.equ("demand")

In [ ]:
# Display the quantities and marginals (=shadow prices) of the demand balance
# constraints in the "detroit" scenario
scen_detroit.equ("demand")

### Close the database connection of the ix modeling platform

In [ ]:
# close the connection of the platform instance to the local ixmp database files
mp.close_db()